# Neural Network Algorithm (The Smarter Algorithm)

### Load The Dataset

In [1]:
import pandas as pd

dataset = pd.read_csv("datasets/heart.csv")
data = dataset.copy()

### Seperate Inputs Features (X) and Target Column(y)

In [2]:
X = data.drop(['HeartDisease'], axis=1)
y = data['HeartDisease']

### Define the Categorical & Numerical Columns

In [3]:
numerical = ["Age","RestingBP","Cholesterol","RestingBP","MaxHR","Oldpeak"]
categorical = ["Sex","ChestPainType","RestingECG","ExerciseAngina","ST_Slope","FastingBS"]

### Scale Numericals and Encode Categorical Columns

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numerical),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical )
])

### Split The Data into Training & Validation Sets

In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(
    X,y,test_size=0.2,stratify=y,random_state=42
)

### Preprocess Data

In [7]:
X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep = preprocessor.fit_transform(X_test)

### Building The Neural Network Model with Hyperparameter s

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    model = Sequential()
    model = model.add(Dense(
        hp.Int("units_1", min_value=32,max_value=128,step=16),
        activation="relu",
        input_shape=(X_test_prep.shape[1],)
    ))
    model.add(Dropout(hp.Float("dropout_1", min_value=0.2,max_value=0.5, step=0.1)))

    model.add(Dense(
        hp.Int("units_2", min_value=16, max_value=64,step=16),
        activation="relu"
    ))

    model.add(Dropout(hp.Float("dropout_2", min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Dense(1, activation="sigmoid"))
    
    model.compile(
        optimizer = Adam(hp.Float("lr", min_value=le-2,sampling="log")),
        loss = "binary_crossentropy",
        matrics = ["accuracy"]
    )

    return model

### Search Tuner

In [15]:
import keras_tuner as kt

tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory="tuner_logs",
    project_name="heart_disease_nn"
)

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AttributeError: 'NoneType' object has no attribute 'add'